# Intro
@Author: Tung Nguyen

@Version: 1.2

This lib is to handle movielens dataset for different deep learning model. Its main task is to create train, test, eval set from RATINGS 

## Esentially we convert dataframe to svm_light, libsvm, libfm format

## And Negative sampling for rating with unified distrubution: 1+:2-




## Import section

In [1]:
#@title
from sklearn.datasets import dump_svmlight_file
import numpy as np
import pandas as pd
import os
import urllib
import zipfile
from sklearn.model_selection import train_test_split
import shutil

In [2]:
#@title
datasets = {'ml100k':'http://files.grouplens.org/datasets/movielens/ml-100k.zip',
            'ml20m':'http://files.grouplens.org/datasets/movielens/ml-20m.zip',
            'mllatestsmall':'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip',
            'ml10m':'http://files.grouplens.org/datasets/movielens/ml-10m.zip',
            'ml1m':'http://files.grouplens.org/datasets/movielens/ml-1m.zip'
            }

# Select dataset to process

## Note: you need to type in your dataset in the input box

In [3]:
print('Available datasets: ', [key for key in datasets])
dt = input('Dataset name = ')
print('You selected {}'.format(dt))

Available datasets:  ['ml100k', 'ml20m', 'mllatestsmall', 'ml10m', 'ml1m']
Dataset name = ml100k
You selected ml100k


In [7]:
dt_dir_name ='C:/Users/thinguyen/Desktop/PhD_2020/Python Code/GNN/Movielens_reprocessing/ml-100k'

## Download and extract

In [4]:
# Download the file from `url` and save it locally under `file_name`:
dt_name = os.path.basename(datasets[dt])

print('Downloading {}'.format(dt_name))
with urllib.request.urlopen(datasets[dt]) as response, open('C:/Users/thinguyen/Desktop/PhD_2020/Python Code/GNN/Movielens_reprocessing/'+dt_name, 'wb') as out_file:
    shutil.copyfileobj(response, out_file)
print('Download completed')

Download completed


In [0]:
with zipfile.ZipFile('./sample_data/'+dt_name, 'r') as zip_ref:
    zip_ref.extractall('./sample_data/')
dt_dir_name = os.path.splitext(dt_name)[0]

In [0]:
#Check unzipped structure
def list_files(startpath):
    print(startpath)
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))
dirs = [x[0] for x in os.walk("./sample_data")]
#print(list(dirs[0]))
ml = filter(lambda dirName: dirName if ('ml' in dirName) else '', list(dirs))
dt_dir_name= list(ml)[0]
print(dt_dir_name)
#list_files('./sample_data/'+dt_dir_name +'/')

./sample_data/ml-1m


## Manual section

You may need to select the data file to check the format

In [8]:
# lets explore movies.csv
if dt=='ml100k':
  rdata= pd.read_csv(dt_dir_name +'/'+ 'u.data', delimiter='\t', names=['userId', 'movieId', 'rating', 'timestamp'])
  usrdata= pd.read_csv(dt_dir_name +'/'+ 'u.user', delimiter='|', names=['user id', 'age' ,'gender' ,'occupation' , 'zip code'])
  rdata.shape
elif dt=='mllatestsmall':
  rdata=pd.read_csv(dt_dir_name +'/'+ 'ratings.csv')#, delimiter='\t'), names=['userid', 'movieId', 'rating', 'timestamp'])  
  tagdata=pd.read_csv(dt_dir_name +'/'+ 'tags.csv')
  rdata.shape
elif dt=='ml1m':
  rdata=pd.read_csv(dt_dir_name +'/'+ 'ratings.dat', delimiter='\:\:', names=['userId', 'movieId', 'rating', 'timestamp'])  
  
elif dt=='ml20m':
  rdata=pd.read_csv(dt_dir_name +'/'+ 'ratings.csv')#, delimiter=',', names=['userid', 'movieId', 'rating', 'timestamp'])  
  tagdata=pd.read_csv(dt_dir_name +'/'+ 'tags.csv')
elif dt=='ml10m':
  rdata=pd.read_csv(dt_dir_name +'/'+ 'ratings.dat', delimiter='\:\:', names=['userId', 'movieId', 'rating', 'timestamp'])

#Optional: Check rating data



In [9]:
rdata.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [10]:
rdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
userId       100000 non-null int64
movieId      100000 non-null int64
rating       100000 non-null int64
timestamp    100000 non-null int64
dtypes: int64(4)
memory usage: 3.1 MB


In [11]:
#is any row null
rdata.isnull().any()

userId       False
movieId      False
rating       False
timestamp    False
dtype: bool

## Check user data

In [12]:
usrdata.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [13]:
usrdata.isnull().any()

user id       False
age           False
gender        False
occupation    False
zip code      False
dtype: bool

## Check tag data

If available (latest)

In [14]:
tagdata.head()

NameError: name 'tagdata' is not defined

In [15]:
tagdata.isnull().any()

NameError: name 'tagdata' is not defined

#Optional: Negative Sampling set
## Can optimize for faster generation 

**(Check out V.1.1 below)**

In [0]:
#This is a slow version. Use v.1.1 instead (below)
# import random
# def neg_sampling(ratings):
#   """version 1.0
#     input rating data: u|m|r
#     return negative sampled set: 1 positive 2 neg (3 time bigger than the original dataset)
#   """
#   users = ratings['userId'].drop_duplicates() #get list uid
#   movies = ratings['movieId'].drop_duplicates() #get list mid
#   nsamples = ratings[['userId', 'movieId']]
#   # nsamplesdict = {[u, i]: r for row in ratings}
#   nsamples['interact'] = nsamples.apply(lambda row: 1, axis=1)

#   for i, row in ratings.iterrows():
#     if(i%1000==0):
#       print("processed ... {}% ...".format(float(i) * 100 / len(ratings)))
#     u=row['userId']
#     j = 2
#     while j > 0:
#       m= movies.sample(n=1).values[0] #get random movie id
#       # insert 2 negative samples if u-m not exist 
#       if(not ((nsamples['userId']==u) & (nsamples['movieId']==m)).any()):
#           j -= 1
#           nsamples.loc[len(nsamples)] = [u, m, -1]
#     #print(nsamples.tail(3))
          
#   return nsamples

## Negative Sampling V1.1 (faster)

In [16]:
#Version 1.1
import random
import time

def neg_sampling(ratings):
  """version 1.1:1 positive 2 neg (3 times bigger than the original dataset)

    Parameters:
    input rating data as pandas dataframe: userId|movieId|rating

    Returns:
    negative sampled set as pandas dataframe
            userId|movieId|interact (implicit)
  """
  users = ratings['userId'].drop_duplicates() #get list uid
  movies = ratings['movieId'].drop_duplicates() #get list mid
  nsamples = ratings[['userId', 'movieId', 'timestamp']]
  nsamples['interact'] = nsamples.apply(lambda row: 1, axis=1)
  nTempData = []
  start_time = time.time()
  stop_time = time.time()
  for i, row in ratings.iterrows():
    if(i%5000==0):
      stop_time = time.time()
      print("processed ... {0:0.2f}% ...{1:0.2f}secs".format(float(i)*100 / len(ratings), stop_time - start_time))
      start_time = stop_time
    u=row['userId']
    j = 2
    while j > 0:
      m= movies.sample(n=1).values[0] #get random movie id
      # insert 2 negative samples if u-m not exist 
      if(not ((nsamples['userId']==u) & (nsamples['movieId']==m)).any()):
          j -= 1
          nTempData.append([u, m, int(time.time()), -1])

  ntempdf =  pd.DataFrame(data=nTempData, columns=['userId', 'movieId', 'timestamp', 'interact' ])      
  nsamples = pd.concat([nsamples, ntempdf],ignore_index=True) 
  return nsamples

In [21]:
#get negative samples to ns var. Later can save it to train/test/. The generation data is quite slow at version 1.
ns = neg_sampling(rdata)
print("done: ", ns.shape)

processed ... 0.00% ...0.00secs
processed ... 5.00% ...19.34secs
processed ... 10.00% ...18.96secs
processed ... 15.00% ...19.38secs
processed ... 20.00% ...19.27secs
processed ... 25.00% ...20.23secs
processed ... 30.00% ...20.11secs
processed ... 35.00% ...19.82secs
processed ... 40.00% ...19.17secs
processed ... 45.00% ...21.61secs
processed ... 50.00% ...21.25secs
processed ... 55.00% ...19.32secs
processed ... 60.00% ...20.57secs
processed ... 65.00% ...20.44secs
processed ... 70.00% ...19.62secs
processed ... 75.00% ...19.60secs
processed ... 80.00% ...19.63secs
processed ... 85.00% ...20.17secs
processed ... 90.00% ...20.61secs
processed ... 95.00% ...18.97secs
done:  (300000, 4)


In [23]:
ns.tail(5)


,userId,movieId,timestamp,interact
299995,276,441,1582191388,-1
299996,13,1664,1582191388,-1
299997,13,1582,1582191388,-1
299998,12,457,1582191388,-1
299999,12,130,1582191388,-1


In [24]:
ns_2 =  pd.DataFrame(data=ns, columns=['userId', 'movieId', 'interact', 'timestamp' ])      

In [25]:
ns_2.tail(5)

,userId,movieId,interact,timestamp
299995,276,441,-1,1582191388
299996,13,1664,-1,1582191388
299997,13,1582,-1,1582191388
299998,12,457,-1,1582191388
299999,12,130,-1,1582191388


In [43]:
# Check the train set
implicit_rating=rdata[['userId', 'movieId', 'rating','timestamp']]
implicit_rating.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [44]:
implicit_rating['interact']= 1

In [45]:
implicit_rating.head()

,userId,movieId,rating,timestamp,interact
0,196,242,3,881250949,1
1,186,302,3,891717742,1
2,22,377,1,878887116,1
3,244,51,2,880606923,1
4,166,346,1,886397596,1


In [46]:
implicit_rating=pd.DataFrame(data=implicit_rating, columns=['userId', 'movieId', 'interact','timestamp'])

In [47]:
implicit_rating.head()

,userId,movieId,interact,timestamp
0,196,242,1,881250949
1,186,302,1,891717742
2,22,377,1,878887116
3,244,51,1,880606923
4,166,346,1,886397596


In [48]:
pd.DataFrame(implicit_rating).to_csv('implicit_rating.csv',encoding='utf-8', index=False, header=False)

In [50]:
ml100k_ns=pd.concat([ns_2,implicit_rating])

In [52]:
ml100k_ns.head()

,userId,movieId,interact,timestamp
0,196,242,1,881250949
1,186,302,1,891717742
2,22,377,1,878887116
3,244,51,1,880606923
4,166,346,1,886397596


In [54]:
ml100k_ns.to_csv('ml100k_ns.csv',encoding='utf-8', index=False, header=False)
#invo_tbl.to_csv(data_path+'invocation_data_2.csv', encoding='utf-8', index=False, header=False)


In [55]:
implicit_rating.to_csv('implicit_rating.csv',encoding='utf-8', index=False, header=False)

## Create train test eval sets for ORIGINAL RATINGS only!!!

In ml100k, there are some train/test sets avaiables

In [0]:
#From rating, drop timestamp

rX_train, rX_test = train_test_split(rdata[['userId', 'movieId', 'rating']], test_size=0.2, random_state = 101)
rX_train, rX_val = train_test_split(rX_train, test_size=0.2, random_state= 101)
print(rX_train.shape, rX_test.shape, rX_val.shape)

(51200, 3) (16000, 3) (12800, 3)


In [0]:
# Check the train set
rX_train.head()

,userId,movieId,rating
73060,473,1895,2.0
63194,414,2671,4.0
67862,438,3968,2.5
25263,177,3624,3.0
100192,610,45210,4.0


## Create train test eval sets for Negative sampling from RATINGS only!!!

In [0]:
rX_train_ns, rX_test_ns = train_test_split(ns, test_size=0.2, random_state = 101)
rX_train_ns, rX_val_ns = train_test_split(rX_train_ns, test_size=0.2, random_state= 101)
print(rX_train_ns.shape, rX_test_ns.shape, rX_val_ns.shape)

(192000, 4) (60000, 4) (48000, 4)


In [0]:
# Check the train set ns
rX_train_ns.head()

,userId,movieId,interact
89651,315,202,1
51626,716,238,1
267896,863,142,-1
16601,418,301,1
183748,203,1053,-1


# Save processed train, test, eval data frame to files



In [0]:
def save_to_csv(data_frames, out_file_names, outdir='./sample_data/processed/'):
  if not os.path.exists(outdir):
    os.mkdir(outdir)
  assert len(data_frames)==len(out_file_names), "number of dataframes must equal number of file names"
  for i in range(len(out_file_names)):
    data_frames[i].to_csv(outdir+out_file_names[i], header=None, index=False)

In [0]:
#Train, test, validation of original data as CSV
save_to_csv([rX_train, rX_test, rX_val], ['rX_train.csv', 'rX_test.csv', 'rx_val.csv'] )

##Train, test, validation of neg_sampling data as CSV

In [0]:
save_to_csv([rX_train_ns, rX_test_ns, rX_val_ns], ['rX_train_ns.csv', 'rX_test_ns.csv', 'rX_val_ns.csv'] )

# Transform train, test, eval to libfm, svmlight, libsvm format as *.libfm

In [0]:
def save_to_libfm(data_frames, out_file_names, target='rating', outdir='./sample_data/processed/'):
  if not os.path.exists(outdir):
    os.mkdir(outdir)
  assert len(data_frames)==len(out_file_names), "number of dataframes must equal number of file names"
  for i in range(len(out_file_names)):
    dump_svmlight_file(data_frames[i][['userId', 'movieId', 'timestamp']],data_frames[i][[target]].values.ravel(), outdir+out_file_names[i],zero_based=True,multilabel=False)

In [0]:
#orginal data
save_to_libfm([rX_train, rX_test, rX_val], ['rX_train.libfm', 'rX_test.libfm', 'rx_val.libfm'])

In [0]:
#negative sampled data
save_to_libfm([rX_train_ns, rX_test_ns, rX_val_ns], ['rX_train_ns.libfm', 'rX_test_ns.libfm', 'rx_val_ns.libfm'], target='interact')

## Zip processed folder and download to local drive

In [0]:
import shutil
zipFileName = 'ml_processed'
shutil.make_archive(zipFileName, 'zip', outdir)

'/content/ml_processed.zip'

In [0]:
#download file to local system
from google.colab import files
files.download(zipFileName+'.zip')

#Or save to GG drive

path =./ in drive

In [0]:
# Import PyDrive and associated libraries.
# This only needs to be done once in a notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a text file.
uploaded = drive.CreateFile({'title': zipFileName+'.zip'})
uploaded.SetContentFile(zipFileName+'.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

##References

Libfm applicatons: https://support.sas.com/resources/papers/proceedings17/SAS0388-2017.pdf

Analysis ML: https://www.kaggle.com/jneupane12/analysis-of-movielens-dataset-beginner-sanalysis

LibSVM: https://www.csie.ntu.edu.tw/~cjlin/libsvm/faq.html#f402

: How do I get the decision value(s)?

We print out decision values for regression. For classification, we solve several binary SVMs for multi-class cases. You can obtain values by easily calling the subroutine svm_predict_values. Their corresponding labels can be obtained from svm_get_labels. Details are in README of libsvm package.

If you are using MATLAB/OCTAVE interface, svmpredict can directly give you decision values. Please see matlab/README for details.

We do not recommend the following. **But if you would like to get values for TWO-class classification with labels +1 and -1 (note: +1 and -1 but not things like 5 and 10) in the easiest way, simply add** 

		printf("%f\n", dec_values[0]*model->label[0]);
after the line
		svm_predict_values(model, x, dec_values);
of the file svm.cpp. Positive (negative) decision values correspond to data predicted as +1 (-1).

### Example
df = pd.DataFrame()
df['Id'] = np.arange(10)
df['F1'] = np.random.rand(10,)
df['F2'] = np.random.rand(10,)
df['Target'] = list(map(lambda x: -1 if x < 0.5 else 1, np.random.rand(10,)))

X = df[np.setdiff1d(df.columns,['Id','Target'])]
y = df.Target

print(df)
